In [1]:
import duckdb

In [ ]:
with duckdb.connect("consumer/data/staging.db", read_only=True) as conn:

    conn.sql("SELECT * FROM shipments").show()

┌─────────────┬──────────┬────────────┬────────────┬─────────────────┬─────────────────┬──────────────┬────────────────────────┬──────────────────────┬───────────────┬─────────┬───────────────┬───────────────┬───────────────────┬────────────────┐
│ shipment_id │ order_id │ carrier_id │ service_id │ tracking_number │ shipping_status │ shipped_date │ expected_delivery_date │ actual_delivery_date │ shipping_cost │   op    │ emitted_ts_ms │     ts_ms     │ connector_version │ transaction_id │
│    int32    │  int32   │   int32    │   int32    │     varchar     │     varchar     │     date     │          date          │         date         │ decimal(10,2) │ varchar │     int64     │     int64     │      varchar      │    varchar     │
├─────────────┼──────────┼────────────┼────────────┼─────────────────┼─────────────────┼──────────────┼────────────────────────┼──────────────────────┼───────────────┼─────────┼───────────────┼───────────────┼───────────────────┼────────────────┤
│           

In [5]:
conn.close()